In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import string


In [2]:
data = pd.read_csv('IMDB Dataset.csv', encoding='utf-8')

In [3]:
class TextPreprocessor:
    def __init__(self, language='english', custom_stopwords=None):
        """
        Initialize the TextPreprocessor class with the specified language for stopwords.
        :param language: The language of stopwords to use (default is 'english').
        """
        nltk.download('stopwords')
        nltk.download('punkt')
        nltk.download('wordnet')
        nltk.download('punkt_tab')

        self.stop_words = set(stopwords.words(language))
        if custom_stopwords:
            self.stop_words.update(custom_stopwords)
        self.lemmatizer = WordNetLemmatizer()
        self.punctuation = set(string.punctuation)

    def preprocess(self, text):
        """
        Preprocess the input text by lowering case, removing stopwords, and lemmatizing words.
        :param text: The text to preprocess.
        :return: A string containing the preprocessed text.
        """
        # Lowercase the text
        text = text.lower()

        # Tokenize the text
        words = word_tokenize(text)

        # Remove stopwords and lemmatize
        words = [self.lemmatizer.lemmatize(word) for word in words if
                 word.isalpha() and word.lower() not in self.stop_words]

        return ' '.join(words)

In [4]:
preprocessor = TextPreprocessor(custom_stopwords=['br'])  # Удаление тегов <br>
data['processed_review'] = data['review'].apply(preprocessor.preprocess)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ilyar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ilyar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ilyar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ilyar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
data['processed_review'][0]

'one reviewer mentioned watching oz episode hooked right exactly happened first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far would say main appeal show due fact go show would dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal could say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison experience watching oz

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data['processed_review'], data['sentiment'], test_size=0.2,
                                                    random_state=42)

In [7]:
# Преобразование текстовых данных в векторы TF-IDF
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Обучение модели логистической регрессии
clf = LogisticRegression(max_iter=100)
clf.fit(X_train_tfidf, y_train)

# Преобразование тестовых данных в векторы TF-IDF
X_test_counts = vectorizer.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

# Предсказание меток для тестовых данных
y_pred = clf.predict(X_test_tfidf)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.89
              precision    recall  f1-score   support

    negative       0.91      0.88      0.89      4961
    positive       0.88      0.91      0.90      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

[[4359  602]
 [ 457 4582]]
